In [1]:
from IPython.display import display
import os

# 上傳檔案（會跳出檔案選擇器）
from ipywidgets import FileUpload

upload = FileUpload()
display(upload)

FileUpload(value={}, description='Upload')

In [2]:
import os
from pathlib import Path

# 假設你只上傳了一個檔案
# Check if any file is uploaded and get the filename
if upload.value:
    # Get the first (and likely only) filename from the dictionary keys
    filename = list(upload.value.keys())[0]
    fileinfo = upload.value[filename] # Access the file info dictionary using the filename as key
else:
    print("No file uploaded.")
    # Handle the case where no file is uploaded, perhaps by exiting or prompting the user.
    # For this example, we'll assume a file was uploaded as per the traceback context.
    raise FileNotFoundError("No file was uploaded.")


# 顯示內容結構（除錯用）
print(fileinfo)

# 儲存 kaggle.json
# filename is already obtained above
content = fileinfo['content']

kaggle_dir = Path.home() / ".kaggle"
kaggle_dir.mkdir(exist_ok=True)

kaggle_json_path = kaggle_dir / "kaggle.json"
with open(kaggle_json_path, "wb") as f:
    f.write(content)

# 設定權限（Linux/macOS 建議）
os.chmod(kaggle_json_path, 0o600)

print(f"{filename} 已成功儲存至 {kaggle_json_path}")

{'metadata': {'name': 'kaggle.json', 'type': 'application/json', 'size': 64, 'lastModified': 1745215961986}, 'content': b'{"username":"suchiwen","key":"01a925cee9e9e9d232008524b0434fb9"}'}
kaggle.json 已成功儲存至 /root/.kaggle/kaggle.json


In [3]:
!kaggle datasets list -s cifar


ref                                                     title                                                  size  lastUpdated                 downloadCount  voteCount  usabilityRating  
------------------------------------------------------  ----------------------------------------------  -----------  --------------------------  -------------  ---------  ---------------  
fedesoriano/cifar100                                    CIFAR-100 Python                                  168517809  2020-12-26 08:37:10.143000          12526        178  1.0              
pankrzysiu/cifar10-python                               CIFAR-10 Python                                   340613496  2018-01-27 13:42:40.967000          15068        256  0.75             
petitbonney/cifar10-image-recognition                   CIFAR-10                                         1007971063  2019-10-01 12:50:23.227000           2979         27  0.8235294        
valentynsichkar/cifar10-preprocessed                   

In [4]:
!pip install -U kaggle
!pip install --upgrade pandas
import os
import zipfile

# 建立 Kaggle 資料夾
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 下載 Dog Breed Identification 資料集
!kaggle competitions download -c dog-breed-identification --force
!unzip -oq dog-breed-identification.zip -d dog-breed-identification


import pandas as pd
import numpy as np

labels = pd.read_csv('dog-breed-identification/labels.csv')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 84.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cp: cannot stat 'kaggle.json': No such file or directory
 88% 607M/691M [00:03<00:01, 52.8MB/s]
100% 691M/691M [00:03<00:00, 196MB/s] 


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision import models, transforms
from torchvision.transforms import RandomErasing # Import RandomErasing
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import pandas as pd
from PIL import Image
from collections import Counter
from tqdm import tqdm
import os

# ---------- 參數設定 ----------
data_dir = "dog-breed-identification"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
labels_path = os.path.join(data_dir, "labels.csv")
val_ratio = 0.2
batch_size = 32
num_workers = 2
gamma = 1.5
num_epochs = 30
lr = 5e-4



# ---------- 資料增強 ----------
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def cutmix(data, targets, alpha=1.0):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    lam = np.random.beta(alpha, alpha)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = shuffled_data[:, :, bbx1:bbx2, bby1:bby2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size(-1) * data.size(-2)))
    return data, targets, shuffled_targets, lam

def rand_bbox(size, lam):
    W, H = size[2], size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w, cut_h = int(W * cut_rat), int(H * cut_rat)
    cx, cy = np.random.randint(W), np.random.randint(H)
    bbx1, bby1 = np.clip(cx - cut_w // 2, 0, W), np.clip(cy - cut_h // 2, 0, H)
    bbx2, bby2 = np.clip(cx + cut_w // 2, 0, W), np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

# Transformations
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])
tta_transforms = [
    transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]) for _ in range(5)
]
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])


In [10]:
# ---------- Dataset ----------
from torch.utils.data import Dataset, random_split

class DogBreedDataset(Dataset):
    def __init__(self, image_dir, labels_df, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.labels_df = labels_df
        self.breed2idx = {breed: idx for idx, breed in enumerate(sorted(labels_df['breed'].unique()))}
        self.labels_df['label'] = self.labels_df['breed'].map(self.breed2idx)

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        row = self.labels_df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['id'] + ".jpg")
        image = Image.open(img_path).convert("RGB")
        label = row['label']
        if self.transform:
            image = self.transform(image)
        return image, label

class TestDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg")]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, img_name

# ---------- 讀入標籤 ----------
labels_df = pd.read_csv(labels_path)
full_dataset = DogBreedDataset(train_dir, labels_df, transform=train_transform)
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
val_dataset.dataset.transform = val_transform

train_labels = [full_dataset.labels_df.iloc[i]['label'] for i in train_dataset.indices]
label_counts = Counter(train_labels)
sample_weights = [1.0 / label_counts[label] for label in train_labels]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=num_workers)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_dataset = TestDataset(test_dir, transform=val_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Combined Loss: Focal + Label Smoothing
class FocalLabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1, gamma=2.0):
        super().__init__()
        self.smoothing = smoothing
        self.gamma = gamma

    def forward(self, input, target):
        log_probs = torch.nn.functional.log_softmax(input, dim=-1)
        targets = torch.zeros_like(log_probs).scatter_(1, target.unsqueeze(1), 1)
        targets = (1 - self.smoothing) * targets + self.smoothing / input.size(1)
        pt = torch.exp(log_probs)
        loss = -((1 - pt) ** self.gamma) * log_probs * targets
        return loss.sum(dim=1).mean()

In [11]:
# ---------- 模型建立 ----------
def get_model(num_classes):
    model = models.vgg16(pretrained=True)
    for param in model.features.parameters():
        param.requires_grad = False
    model.classifier = nn.Sequential(
        nn.Linear(25088, 4096), nn.ReLU(True), nn.Dropout(0.5),
        nn.Linear(4096, 4096), nn.ReLU(True), nn.Dropout(0.5),
        nn.Linear(4096, num_classes)
    )
    return model

In [12]:
# K-Fold Training + TTA
def train_kfold(dataset, n_splits=5):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"Fold {fold+1}/{n_splits}")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Weighted Sampling
        train_labels = [dataset.labels_df.iloc[i]['label'] for i in train_idx]
        label_counts = Counter(train_labels)
        weights = [1.0 / label_counts[label] for label in train_labels]
        sampler = WeightedRandomSampler(weights, len(weights))

        train_loader = DataLoader(train_subset, batch_size=128, sampler=sampler, num_workers=2)
        val_loader = DataLoader(val_subset, batch_size=128, shuffle=False, num_workers=2)

        model = get_model(len(label_counts)).to(device)
        criterion = FocalLabelSmoothingLoss(smoothing=0.1, gamma=2.0)
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=2)

        best_acc = 0
        patience, counter = 5, 0
        for epoch in range(30):
            model.train()
            total_loss = 0
            for images, labels in tqdm(train_loader):
                images, labels = images.to(device), labels.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            scheduler.step()

            # Validation
            model.eval()
            correct, total = 0, 0
            all_preds, all_targets = [], []
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    preds = outputs.argmax(dim=1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)
                    all_preds.extend(preds.cpu().numpy())
                    all_targets.extend(labels.cpu().numpy())

            val_acc = correct / total
            print(f"Epoch {epoch+1} - Val Accuracy: {val_acc:.4f}")
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), f"best_model_fold{fold}.pth")
                counter = 0
                print("Model saved")
            else:
                counter += 1
                if counter >= patience:
                    print("Early stopping")
                    break

        # Confusion Matrix
        print(confusion_matrix(all_targets, all_preds))
        print(classification_report(all_targets, all_preds))


In [13]:
# Inference with TTA
@torch.no_grad()
def tta_inference(model, image, transforms_list):
    model.eval()
    image = image.to(device)
    preds = torch.zeros((len(class_names),), device=device)
    for tform in transforms_list:
        augmented = tform(image.cpu())
        augmented = augmented.unsqueeze(0).to(device)
        output = model(augmented)
        preds += torch.softmax(output, dim=1).squeeze(0)
    return preds / len(transforms_list)

# Example Usage
labels_df = pd.read_csv("dog-breed-identification/labels.csv")
train_dataset = DogBreedDataset("dog-breed-identification/train", labels_df, transform=train_transform)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_names = sorted(labels_df['breed'].unique())
train_kfold(train_dataset)

Fold 1/5


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 95.7MB/s]
100%|██████████| 64/64 [00:50<00:00,  1.28it/s]


Epoch 1 - Val Accuracy: 0.4132
Model saved


100%|██████████| 64/64 [00:50<00:00,  1.26it/s]


Epoch 2 - Val Accuracy: 0.5604
Model saved


...

Epoch 8 - Val Accuracy: 0.6150


100%|██████████| 64/64 [00:51<00:00,  1.24it/s]


Epoch 9 - Val Accuracy: 0.6243


100%|██████████| 64/64 [00:52<00:00,  1.22it/s]


Epoch 10 - Val Accuracy: 0.6204
Early stopping
[[10  1  0 ...  0  0  1]
 [ 0 15  0 ...  0  0  0]
 [ 0  0 13 ...  0  0  0]
 ...
 [ 0  0  1 ...  9  0  0]
 [ 0  0  0 ...  0 20  0]
 [ 1  0  0 ...  0  0  7]]
              precision    recall  f1-score   support

           0       0.45      0.59      0.51        17
           1       0.88      0.62      0.73        24
           2       0.93      0.87      0.90        15
           3       0.89      0.38      0.53        21
           4       0.56      0.53      0.54        19
           5       0.50      0.45      0.48        11
           6       0.73      0.58      0.65        19
           7       0.60      0.71      0.65        21
           8       0.68      0.65      0.67        20
           9       0.63      0.76      0.69        25
          10       0.76      0.76      0.76        17
          11       0.76      0.62      0.68        21
          12       0.48      0.81      0.60        16
          13       0.59      0.89      0

In [17]:
# ---------- 預測 ----------
# Instantiate the model before loading the state dict
# You need to determine the correct number of classes for the model
# Based on your training code, the number of classes is derived from the unique breeds in labels_df
labels_df = pd.read_csv(labels_path) # Make sure labels_df is accessible
num_classes = len(labels_df['breed'].unique())

model = get_model(num_classes).to(device)

# Load the state dictionary of the best model from the K-Fold training
# You might need to change "best_model_fold0.pth" depending on which fold performed best
# or if you implement a different saving strategy during training.
model.load_state_dict(torch.load("best_model_fold0.pth")) # Assuming fold 0 had the best model
model.eval()

all_probs, all_img_ids = [], []
with torch.no_grad():
    for images, img_ids in test_loader:
        images = images.to(device)
        outputs = model(images)
        probs = torch.softmax(outputs, dim=1)
        all_probs.extend(probs.cpu().numpy())
        all_img_ids.extend([img_id.replace(".jpg", "") for img_id in img_ids])

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
# ---------- Submission ----------
class_names = sorted(labels_df['breed'].unique())
submission = pd.DataFrame(all_probs, columns=class_names)
submission.insert(0, "id", all_img_ids)
submission.to_csv("submission.csv", index=False)
print("✅ submission.csv 已儲存")

✅ submission.csv 已儲存


In [72]:
!pip freeze

absl-py==1.4.0
accelerate==1.7.0
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.1
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.9.0
argon2-cffi==25.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.2
arviz==0.21.0
astropy==7.1.0
astropy-iers-data==0.2025.6.2.0.38.23
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
backports.tarfile==1.2.0
beautifulsoup4==4.13.4
betterproto==2.0.0b6
bigframes==2.5.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.0.0
blosc2==3.3.4
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.1
build==1.2.2.post1
CacheControl==0.14.3
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.4.26
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.2
chex==0.1.89
clarabel==0.11.0
click==8.2.1
cloudpathlib==0.21.1
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy

In [73]:
import torch
print("GPU 是否可用：", torch.cuda.is_available())
print("可用 GPU 數量：", torch.cuda.device_count())
print("GPU 名稱：", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "無")

GPU 是否可用： True
可用 GPU 數量： 1
GPU 名稱： Tesla T4
